In [ ]:
from flask import Flask, request, jsonify
import whisper
import os
import torch
import soundfile as sf

MODEL_NAME = "large-v3-turbo"  # 필요에 따라 변경 가능
LANGUAGE = "ko"                # "auto" 가능
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model(MODEL_NAME, device=DEVICE)
fp16 = bool(torch.cuda.is_available())
app = Flask(__name__)

def get_audio_duration(file_path):
    try:
        f = sf.SoundFile(file_path)
        return f.frames / f.samplerate
    except Exception:
        return None

def has_mostly_foreign(text):
    import re
    # 한글, 숫자, 공백, 기본문장부호 이외 많이 들어가면 foreign으로 판단
    return re.fullmatch(r"[\s,.·a-zA-Z0-9ㄱ-ㅎ가-힣?!~\"'‘’“”\(\)\[\]\-…]+", text) is None

@app.route('/stt', methods=['POST'])
def stt():
    json_data = request.get_json()
    if not json_data or 'file_path' not in json_data:
        return jsonify({"error": "file_path가 필요합니다"}), 400

    file_path = json_data['file_path']

    if not os.path.isfile(file_path):
        return jsonify({"error": "파일이 존재하지 않습니다"}), 400

    audio_duration = get_audio_duration(file_path)  # 초 단위 float

    prompt_for_shorter_segments = "사용자는 매우 짧고 간결한 문장으로 말하고 있습니다. 쉼표(,), 마침표(.)를 자주 사용하세요."
    result = model.transcribe(
        file_path,
        language=None if LANGUAGE == "auto" else LANGUAGE,
        fp16=fp16,
        verbose=True,
        initial_prompt=prompt_for_shorter_segments 
    )
    text = result.get("text", "").strip()
    segments = result.get("segments", [])

    # 타임라인 있는 리스트 생성 (start, end, text만 추출)
    timeline = []
    for seg in segments:
        s = seg["start"]
        e = seg["end"]
        t = seg["text"].strip()
        # 1. end가 오디오 길이보다 크면 무시
        # 2. 외국어/잡음 구간(짧거나 has_mostly_foreign)이면 무시
        if audio_duration and e > audio_duration + 1.0:
            continue
        if len(t) < 2 or has_mostly_foreign(t):
            continue
        timeline.append({
            "start": s,
            "end": e,
            "text": t
        })

    return jsonify({
        "text": text,
        "timeline": timeline
    })

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5002)